In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from joblib import dump

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn import svm

In [4]:
df = pd.read_csv('../data/raw/train.csv')
df.head()

,Id_old,Id,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,10556,3799,80,24.3,7.8,3.0,6.4,45.7,0.1,0.3,...,2.9,72.1,2.2,2.0,3.8,3.2,1.1,0.2,1.6,1
1,5342,3800,75,21.8,10.5,4.2,7.9,55.1,-0.3,-1.0,...,3.6,67.8,3.6,3.7,6.6,0.7,0.5,0.6,1.4,1
2,5716,3801,85,19.1,4.5,1.9,4.5,42.8,0.4,1.2,...,0.6,75.7,0.6,1.8,2.4,0.8,0.4,0.2,0.6,1
3,13790,3802,63,19.1,8.2,3.5,6.7,52.5,0.3,0.8,...,1.5,66.9,0.8,2.0,3.0,1.8,0.4,0.1,1.9,1
4,5470,3803,63,17.8,3.7,1.7,3.4,50.8,0.5,1.4,...,0.5,54.0,2.4,2.7,4.9,0.4,0.4,0.6,0.7,1


In [5]:
cols_drop = ["Id", "Id_old", "MIN", "FGM", "FGA", 'TOV', '3PA', 'FTM', 'FTA', 'REB']
df.drop(cols_drop, axis=1, inplace=True)

In [6]:
df.head()

,GP,PTS,FG%,3P Made,3P%,FT%,OREB,DREB,AST,STL,BLK,TARGET_5Yrs
0,80,7.8,45.7,0.1,22.6,72.1,2.2,2.0,3.2,1.1,0.2,1
1,75,10.5,55.1,-0.3,34.9,67.8,3.6,3.7,0.7,0.5,0.6,1
2,85,4.5,42.8,0.4,34.3,75.7,0.6,1.8,0.8,0.4,0.2,1
3,63,8.2,52.5,0.3,23.7,66.9,0.8,2.0,1.8,0.4,0.1,1
4,63,3.7,50.8,0.5,13.7,54.0,2.4,2.7,0.4,0.4,0.6,1


In [7]:
df_cleaned = df.copy()
# Strip the column names if they contain spaces
df_cleaned.columns = df_cleaned.columns.str.strip()

# Extract the target column
target = df_cleaned.pop('TARGET_5Yrs')

# Scaling
# scaler = StandardScaler()
# df_cleaned = scaler.fit_transform(df_cleaned)
#df_cleaned

X_train, X_val, y_train, y_val = train_test_split(df_cleaned, target, test_size = 0.2, random_state=8 )

# Save the splited data
np.save('../data/processed/X_train', X_train)
np.save('../data/processed/X_val', X_val)

np.save('../data/processed/y_train', y_train)
np.save('../data/processed/y_val', y_val)

In [162]:
X_train.shape

(6400, 11)

In [182]:
# model = DecisionTreeClassifier(max_depth = 10)
# model.fit(X_train,y_train )

model = svm.SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
y_train_prediction = model.predict(X_train)
y_train_prediction
confusion_matrix(y_train, y_train_prediction) # 
accuracy_score(y_train, y_train_prediction) #

NameError: name 'model' is not defined

In [157]:
y_val_prediction = model.predict(X_val)
y_val_prediction
#confusion_matrix(y_val, y_val_prediction)
accuracy_score(y_val, y_val_prediction) #0.81875

0.839375

In [48]:
df_test = pd.read_csv('../data/raw/test.csv')
df_test.head()

#df_test = df_test.iloc[:, 2:]
cols_drop = ["Id", "Id_old", "MIN", "FGM", "FGA", 'TOV', '3PA', 'FTM', 'FTA', 'REB']
df_test.drop(cols_drop, axis=1, inplace=True)
df_test.head(5)
#df_test.shape

,GP,PTS,FG%,3P Made,3P%,FT%,OREB,DREB,AST,STL,BLK
0,56,4.0,43.7,0.1,7.3,63.4,1.2,0.8,0.4,0.2,0.3
1,43,10.1,46.0,0.6,35.1,75.3,0.5,0.9,3.5,0.6,0.0
2,82,11.3,45.6,0.5,44.8,71.2,1.3,3.3,2.5,1.3,0.3
3,86,18.8,42.9,0.5,13.5,70.9,1.5,3.2,4.1,0.9,0.1
4,58,4.7,40.0,0.5,38.7,76.9,0.2,0.6,1.5,0.5,-0.4


In [49]:
df_test_cleaned = df_test.copy()

# Strip the column names if they contain spaces
df_test_cleaned.columns = df_test_cleaned.columns.str.strip()

# scaler = StandardScaler()
# df_test_cleaned = scaler.fit_transform(df_test_cleaned)

df_test_cleaned.shape
# y_test_prediction = model.predict(df_test_cleaned)

# y_test_prediction

# final_prediction_test = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p for p in y_test_prediction]})
# final_prediction_test.groupby('TARGET_5Yrs').sum()

#final_prediction_test.to_csv("../reports/tin_submission_01.csv", index=False)

(3799, 11)

In [168]:
final_prediction_test.groupby('TARGET_5Yrs').sum()

,Id
TARGET_5Yrs,
1,7214301


In [104]:
final_prediction_test.groupby('TARGET_5Yrs').sum()
final_prediction_test.to_csv("../reports/tin_submission_01.csv", index=False)
# https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python
# https://stackabuse.com/classification-in-python-with-scikit-learn-and-pandas/
#https://machinelearningmastery.com/types-of-classification-in-machine-learning/
#https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python

In [62]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)

from sklearn.linear_model import LogisticRegression 
log_reg = LogisticRegression().fit(X_train_poly, y_train)
# dump(lm_reg, '../models/linear_regression_default.joblib')

C:\Hnin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [81]:
X_val_poly = poly.fit_transform(X_val)
# y_pred=log_reg.predict_proba(X_val_poly)[:,1]
# y_pred

y_pred=log_reg.predict(X_val_poly)
y_pred
accuracy_score(y_val, y_pred) #0.836875

0.836875

In [82]:
df_test_cleaned.head()

# poly = PolynomialFeatures(2)
X_test_poly = poly.fit_transform(df_test_cleaned)

y_pred=log_reg.predict_proba(X_test_poly)[:,1]
y_pred
final_prediction_test = pd.DataFrame({'Id': range(0,3799), 'TARGET_5Yrs': [p for p in y_pred]})
#final_prediction_test.groupby('TARGET_5Yrs').sum()

final_prediction_test.to_csv("../reports/tin_submission_01.csv", index=False)
final_prediction_test

,Id,TARGET_5Yrs
0,0,0.829444
1,1,0.750505
2,2,0.952749
3,3,0.957862
4,4,0.753943
...,...,...
3794,3794,0.952477
3795,3795,0.826067
3796,3796,0.750761
3797,3797,0.951689
